In [4]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os
import cv2
import seaborn as sns
from sklearn.model_selection import train_test_split


In [5]:
image_1_path="/kaggle/input/brain-tumor-classification-mri/Training/glioma_tumor"
image_2_path="/kaggle/input/brain-tumor-classification-mri/Training/meningioma_tumor"
image_3_path="/kaggle/input/brain-tumor-classification-mri/Training/pituitary_tumor"
image_4_path="/kaggle/input/brain-tumor-classification-mri/Training/no_tumor"

In [6]:
image_1_data=[]
image_1_storage = [os.path.join(image_2_path,i) for i in os.listdir(image_2_path)]
image_cv = [ cv2.imread(file) for file in image_1_storage]
# image_array=np.array(image_cv)
for image in image_cv:
    if image is not None and image.size > 0:
        resize_image=cv2.resize(image, (250,250))
        image_1_data.append(resize_image)
        image_array=np.array(image_1_data)

In [7]:
image_array.shape

(822, 250, 250, 3)

In [8]:
def image_convert(image_path):
    import numpy as np
    import cv2
    image_1_data=[]
    image_1_storage = [os.path.join(image_path,i) for i in os.listdir(image_path)]
    image_cv = [ cv2.imread(file) for file in image_1_storage]
    for image in image_cv:
        if image is not None and image.size > 0:
            resize_image=cv2.resize(image, (250,250))
            image_1_data.append(resize_image)
            image_array=np.array(image_1_data)
    return image_array

In [9]:
image_1_data1=image_convert(image_1_path)
image_2_data1=image_convert(image_2_path)
image_3_data1=image_convert(image_3_path)
# image_3_data1["Label"]=1
image_4_data1=image_convert(image_4_path)
# image_4_data1["Label"]=0

In [10]:
x=len(image_1_data1)+len(image_2_data1)+len(image_3_data1)
# x

In [11]:
# tumor
# i=1
tumor_label=[]
for i in range(2475):
    tumor_label.append(1)
# len(image_4_data1)
# tumor_label

In [12]:
# no_tumor
i=0
notumor_label=[]
for i in range(395):
    notumor_label.append(i-i)
# len(image_4_data1)
len(notumor_label)


395

In [13]:
label=np.array(tumor_label+notumor_label)
len(label)

2870

In [14]:
label[2470:2480]

array([1, 1, 1, 1, 1, 0, 0, 0, 0, 0])

In [15]:
combined_image_data_con = np.concatenate((image_1_data1, image_2_data1, image_3_data1, image_4_data1), axis=0)
print(combined_image_data_con.shape)
print(len(combined_image_data_con))

(2870, 250, 250, 3)
2870


In [16]:
X=combined_image_data_con
Y=label

In [17]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,random_state=42)

In [18]:
print(X_train.shape)
print(X_test.shape)

(2152, 250, 250, 3)
(718, 250, 250, 3)


# # Deep Neural Network

In [19]:
from keras.layers import Dense,Flatten,Conv2D,MaxPooling2D,Dropout,BatchNormalization,Input
from keras.models import Sequential
from keras.callbacks import EarlyStopping

2024-05-24 08:55:22.554663: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-24 08:55:22.554767: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-24 08:55:22.833759: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Model1

In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Input

# Assuming X_train and X_test are numpy arrays of shape (num_samples, 249, 250, 3)
# Assuming Y_train and Y_test are numpy arrays of shape (num_samples,)

# Create Sequential model
model123 = Sequential([
    Input(shape=(250, 250, 3)),
    Conv2D(32, (3, 3), activation="relu"),
    Conv2D(64, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(64, (5, 5), activation="relu"),
    Conv2D(128, (7, 7), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(512, (5, 5), activation="relu"),
    Conv2D(128, (7, 7), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Conv2D(1024, (7, 7), activation="relu"),
    Conv2D(512, (3, 3), activation="relu"),
    MaxPooling2D(2, 2),
    Dropout(0.25),
    Flatten(),
    Dense(2000, activation="relu"),
    Dense(1000, activation="relu"),
    Dropout(0.25),
    Dense(2500, activation="relu"),
    Dense(500, activation="relu"),
    Dropout(0.25),
    Dense(150, activation="relu"),
    Dense(1, activation="sigmoid")  # Use 'sigmoid' for binary classification
])

# Compile the model
model123.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


In [29]:
early_stopping=EarlyStopping(
    monitor = "val_loss",
    patience = 7,
    restore_best_weights = True
)

In [33]:
model123.fit(X_train,Y_train,epochs=100,batch_size=32,verbose=1,validation_data=(X_test,Y_test),validation_split=0.2,callbacks=[early_stopping])

Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 387ms/step - accuracy: 0.8706 - loss: 0.3980 - val_accuracy: 0.8649 - val_loss: 0.4361
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 379ms/step - accuracy: 0.8582 - loss: 0.4425 - val_accuracy: 0.8649 - val_loss: 0.3990
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 25s 373ms/step - accuracy: 0.8664 - loss: 0.4016 - val_accuracy: 0.8649 - val_loss: 0.3982
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 25s 373ms/step - accuracy: 0.8588 - loss: 0.4087 - val_accuracy: 0.8649 - val_loss: 0.3985
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 25s 374ms/step - accuracy: 0.8609 - loss: 0.4048 - val_accuracy: 0.8649 - val_loss: 0.3961
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 25s 374ms/step - accuracy: 0.8697 - loss: 0.3929 - val_accuracy: 0.8649 - val_loss: 0.4102
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 25s 374ms/step - accuracy: 0.8504 - loss: 0.4328 - val_accuracy: 0.8649 - val_loss: 0.3961


# Accuracy

In [34]:
loss,accuracy=model123.evaluate(X_test,Y_test)
print("Loss of Sequential model:",loss)
print("accuracy of Sequential model:",accuracy)

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.8701 - loss: 0.4304
Loss of Sequential model: 0.43614983558654785
accuracy of Sequential model: 0.8649024963378906


# Model2

In [30]:
model12=Sequential([
    Input(shape=(249, 250, 3)),
    Conv2D(32,(3,3),activation="relu"),
    Conv2D(64,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(64,(5,5),activation="relu"),
    Conv2D(128,(7,7),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(512,(5,5),activation="relu"),
    Conv2D(128,(7,7),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Conv2D(1024,(7,7),activation="relu"),
    Conv2D(512,(3,3),activation="relu"),
    MaxPooling2D(2,2),
    Dropout(0.25),
    Flatten(),
    Dense(2000,activation="relu"),
    Dense(1000,activation="relu"),
    Dropout(0.25),
    Dense(2500,activation="relu"),
    Dense(500,activation="relu"),
    Dropout(0.25),
    Dense(150,activation="relu"),
    Dense(1,activation="sigmoid")
])
model12.compile(loss="binary_crossentropy",optimizer="adam",metrics=["accuracy"])


In [31]:
model12.fit(X_train,Y_train,epochs=100,batch_size=32,verbose=1,validation_data=(X_test,Y_test),validation_split=0.2,callbacks=[early_stopping])

Epoch 1/100


W0000 00:00:1716541694.204108     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


67/68 ━━━━━━━━━━━━━━━━━━━━ 0s 360ms/step - accuracy: 0.7550 - loss: 38.1897

W0000 00:00:1716541729.341088     111 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


68/68 ━━━━━━━━━━━━━━━━━━━━ 0s 522ms/step - accuracy: 0.7560 - loss: 37.7824

W0000 00:00:1716541730.577726     109 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1716541733.022421     110 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


68/68 ━━━━━━━━━━━━━━━━━━━━ 54s 584ms/step - accuracy: 0.7570 - loss: 37.3870 - val_accuracy: 0.8649 - val_loss: 0.4054
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 27s 391ms/step - accuracy: 0.8673 - loss: 0.3958 - val_accuracy: 0.8649 - val_loss: 0.3866
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 384ms/step - accuracy: 0.8494 - loss: 0.4266 - val_accuracy: 0.8649 - val_loss: 0.3977
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 384ms/step - accuracy: 0.8520 - loss: 0.4253 - val_accuracy: 0.8649 - val_loss: 0.3883
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 381ms/step - accuracy: 0.8757 - loss: 0.3756 - val_accuracy: 0.8649 - val_loss: 0.3996
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 380ms/step - accuracy: 0.8565 - loss: 0.4142 - val_accuracy: 0.8649 - val_loss: 0.3980
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 387ms/step - accuracy: 0.8534 - loss: 0.4210 - val_accuracy: 0.8649 - val_loss: 0.3773
Epoch 8/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 26s 382ms/step - accuracy: 0.8640 - loss: 0.4126 - val_accurac

# * > Accuracy

In [35]:
loss,accuracy=model12.evaluate(X_test,Y_test)
print("Loss of Sequential model:",loss)
print("accuracy of Sequential model:",accuracy)

23/23 ━━━━━━━━━━━━━━━━━━━━ 2s 90ms/step - accuracy: 0.8707 - loss: 0.3506
Loss of Sequential model: 0.3606950044631958
accuracy of Sequential model: 0.8662952780723572


# Model3

In [27]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Reshape input data to treat frames as channels
# X_train_reshaped = X_train.reshape(-1, 750, 3)  # Shape: (2478, 750, 3)
# X_test_reshaped=X_test.reshape(-1,750,3)
# Create Sequential model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(250, 250, 3),padding="same"),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu',padding="same"),
    MaxPooling2D(2,2),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
                ])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=100, batch_size=32, verbose=1,validation_data=(X_test,Y_test),validation_split=0.2,callbacks=[early_stopping])
# print(X_train_reshaped.shape)

/opt/conda/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Epoch 1/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 16s 123ms/step - accuracy: 0.1479 - loss: nan - val_accuracy: 0.1351 - val_loss: nan
Epoch 2/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1441 - loss: nan - val_accuracy: 0.1351 - val_loss: nan
Epoch 3/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1352 - loss: nan - val_accuracy: 0.1351 - val_loss: nan
Epoch 4/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1348 - loss: nan - val_accuracy: 0.1351 - val_loss: nan
Epoch 5/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.1406 - loss: nan - val_accuracy: 0.1351 - val_loss: nan
Epoch 6/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1334 - loss: nan - val_accuracy: 0.1351 - val_loss: nan
Epoch 7/100
68/68 ━━━━━━━━━━━━━━━━━━━━ 3s 42ms/step - accuracy: 0.1423 - loss: nan - val_accuracy: 0.1351 - val_loss: nan


#  * Accuracy

In [28]:
loss,accuracy=model.evaluate(X_test,Y_test)
print("Loss of Sequential model:",loss)
print("accuracy of Sequential model:",accuracy)

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.1299 - loss: nan
Loss of Sequential model: nan
accuracy of Sequential model: 0.13509748876094818
